In [ ]:
!pip install pronouncing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pronouncing
import nltk
from nltk.corpus import brown

nltk.download('brown')

freqs = nltk.FreqDist([w.lower() for w in brown.words()])

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [ ]:
wordlist = set(pronouncing.rhymes(""))

wordlist_sorted = sorted(wordlist, key=lambda x: freqs[x.lower()], reverse=True)
# print the sorted list
for w in wordlist_sorted:
    print(w)

In [ ]:
pronouncing.phones_for_word("sentence")

['S EH1 N T AH0 N S']

In [ ]:
import re

regex = re.compile('[^a-zA-Z ]')

In [ ]:
from collections import OrderedDict
from termcolor import colored

def print_colored_rhymes(sentence, more=True, top=-1):
  words = regex.sub('', sentence).split()

  all_syllables = set()
  word_to_phone = dict()
  for word in words:
    word = word.lower()
    try:
      phones = pronouncing.phones_for_word(word)[0].split()
      word_to_phone[word] = phones
      for phone in phones:
        all_syllables.add(phone)
    except:
      pass

  rhymes = OrderedDict()
  for syllable in all_syllables:
    rhymes[syllable] = [k for k, v in word_to_phone.items() if syllable in v]
  rhymes = OrderedDict(sorted(rhymes.items(), key=lambda key_value_pair: len(key_value_pair[1])))
  rhymes = OrderedDict(reversed(list(rhymes.items())))

  colors = ['on_yellow', 'on_magenta', 'on_green', 'on_blue', 'on_grey']

  top_rhymes = []
  if more:
    number_of_rhymes = len(colors) * 2
  else:
    number_of_rhymes = len(colors)
  for i in range(number_of_rhymes):
    if len(rhymes) == 0: break
    top_rhymes.append(rhymes.popitem(last=False))
    if len(top_rhymes[-1][1]) == 1:
      top_rhymes.pop()
      break

  word_to_color = dict()
  for color_index, rhyme in enumerate(top_rhymes):
    if not top == -1 and top-1 != color_index:
      continue
    for word in rhyme[1]:
      word = word.lower()
      for syllable_index in [i for i, n in enumerate(word_to_phone[word]) if n == rhyme[0]]:
        syllable_percent = syllable_index / len(word_to_phone[word])
        syllable_end_percent = (syllable_index + 1) / len(word_to_phone[word])
        syllable_index = round(syllable_percent * len(word))
        syllable_end_index = round(syllable_end_percent * len(word))
        if syllable_index == len(word): syllable_index -= 1
        if syllable_end_index == 0: syllable_end_index = 1

        if color_index == 0:
          text_color = 'grey'
        elif color_index // len(colors) == 0:
          text_color = 'white'
        else:
          text_color = 'red'
        color = colors[color_index % len(colors)]

        if any(word == d for d in word_to_color):
          for i in range(syllable_index, syllable_end_index):
            word_to_color[word][i] = colored(word_to_color[word][i], text_color, color)
        else:
          formattedText = [i for i in word[:syllable_index]]
          formattedText += [colored(i, text_color, color) for i in word[syllable_index:syllable_end_index]]
          formattedText += [i for i in word[syllable_end_index:]]
          word_to_color[word] = formattedText

  count = 0
  if len(top_rhymes) < top:
    return

  for line in sentence.splitlines():
    for item in line.split():
      word = re.sub(r'[^a-zA-Z]', '', item).lower()
      if any(word == d for d in word_to_color):
        count += 1
        print("".join(word_to_color[word]), end=' ')
      else:
        print(item, end=' ')
    print()

  if top != -1:
    print(f'{list(top_rhymes)[top-1][0]} counted {count} times')

In [ ]:
sentence = "Well if you want Shady, then this is what I’ll give ya \n A little bit of weed mixed with some hard liquor \n Some vodka that will jumpstart my heart quicker \n Then a shot when I get shocked at the hospital \n By the doctor when I’m not cooperating."
print_colored_rhymes(sentence)
print()
print_colored_rhymes(sentence, False)
for i in range(1, 6):
  print()
  print_colored_rhymes(sentence, top=i)

well if you want shady then this is what ill give ya 
A little bit of weed mixed with some hard liquor 
some vodka that will jumpstart my heart quicker 
then a shot when I get shocked at the hospital 
By the doctor when im not cooperating 

well if you want Shady, then this is what ill give ya 
A little bit of weed mixed with some hard liquor 
Some vodka that will jumpstart my heart quicker 
Then a shot when I get shocked at the hospital 
By the doctor when im not cooperating 

Well if you want Shady, then this is what I’ll give ya 
A little bit of weed mixed with some hard liquor 
Some vodka that will jumpstart my heart quicker 
Then a shot when I get shocked at the hospital 
By the doctor when I’m not cooperating 
T counted 16 times

Well if you want Shady, then this is what ill give ya 
A little bit of weed mixed with some hard liquor 
Some vodka that will jumpstart my heart quicker 
Then a shot when I get shocked at the hospital 
By the doctor when im not cooperating. 
IH1 counted 

In [ ]:
sentence = 'This is a random sentence that I am not trying to rhyme'
print_colored_rhymes(sentence)
print()
print_colored_rhymes(sentence, False)
print()
print_colored_rhymes(sentence, top=1)

this is a random sentence that i am not trying to rhyme 

This is a random sentence that i am not trying to rhyme 

This is a random sentence that I am not trying to rhyme 
T counted 5 times


In [ ]:
sentence = input('input: ')
print_colored_rhymes(sentence)
print()
print_colored_rhymes(sentence, False)
for i in range(1, 6):
  print()
  print_colored_rhymes(sentence, top=i)

input: a b c d e f g h i j k l m n o p q r s t u v w x y z
a b c d e f g h i j k l m n o p q r s t u v w x y z 

a b c d e f g h i j k l m n o p q r s t u v w x y z 

a b c d e f g h i j k l m n o p q r s t u v w x y z 
IY1 counted 9 times

a b c d e f g h i j k l m n o p q r s t u v w x y z 
EH1 counted 6 times

a b c d e f g h i j k l m n o p q r s t u v w x y z 
K counted 3 times

a b c d e f g h i j k l m n o p q r s t u v w x y z 
S counted 3 times

a b c d e f g h i j k l m n o p q r s t u v w x y z 
Y counted 3 times
